# 光ファイバ要素試験解析 - クイックスタート

Google Colab またはローカル Jupyter で実行できます。

- **単一ケース**: 1 ケース分の理論 vs 換算せん断ひずみの図と指標を `output/<case_id>/` に保存
- **複数ケース比較**: 複数ケースを 1 枚の図にまとめた比較結果を `output/<比較名>/` に保存

図化するケースの指定方法は **3** と **4** の見出しで案内しています。

## 1. 環境準備

**Colab で使う場合**: あらかじめ GitHub から clone したリポジトリを Google Drive の任意のフォルダに置き、このノートブックを Drive 上で「Colab で開く」してから実行してください。下の 1 つ目のセルで **Drive マウント** と **REPO_PATH**（Drive 上のリポジトリパス）を設定し、2 つ目のセルで **pip インストール** のみ行います（clone はノートブック内では行いません）。

**ローカル**: 既存リポジトリのプロジェクトルートを自動判定し、2 つ目のセルで依存確認のみ行います。下の 2 つのセルを順に実行してください。

In [ ]:
import os
from pathlib import Path

# Google Colab 上で実行しているか判定
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
print(f"実行環境: {'Google Colab' if IN_COLAB else 'ローカル / その他'}")

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Drive 上に置いたリポジトリのパス（基本はこのパスで統一。必要なら変更）
    REPO_PATH = "/content/drive/MyDrive/DFOS-Element-Tests"
    PROJECT_ROOT = Path(REPO_PATH)
    if not PROJECT_ROOT.is_dir():
        print(f"警告: REPO_PATH が存在しません: {REPO_PATH}")
    os.chdir(PROJECT_ROOT)
    print(f"プロジェクトルート: {PROJECT_ROOT}")
    if PROJECT_ROOT.is_dir():
        print("中身:")
        for p in sorted(PROJECT_ROOT.iterdir()):
            print(f"  - {p.name}/" if p.is_dir() else f"  - {p.name}")
else:
    PROJECT_ROOT = Path.cwd() if (Path.cwd() / "params").is_dir() else Path.cwd().parent
    os.chdir(PROJECT_ROOT)
    print(f"プロジェクトルート: {PROJECT_ROOT}")

実行環境: ローカル / その他
プロジェクトルート: /srv/shared_drive/DFOS-Element-Tests


In [2]:
import shutil
import subprocess
import sys

if IN_COLAB:
    req = "requirements-colab.txt" if (PROJECT_ROOT / "requirements-colab.txt").exists() else "requirements.txt"
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", req], check=True, cwd=PROJECT_ROOT)
    print("インストール完了（Colab）")
else:
    if shutil.which("uv"):
        subprocess.run(["uv", "pip", "install", "-q", "-r", "requirements.txt"], check=True)
    else:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"], check=True)
    print("ローカル: 既存リポジトリを使用し、依存パッケージを確認しました。")

ローカル: 既存リポジトリを使用し、依存パッケージを確認しました。


## 2. データ・出力先の設定（Colab 時は任意で Drive マウント）

Colab で自分のデータや結果を Drive に置く場合だけ、Drive マウントのセルを実行してください。ローカルでは `data/` と `output/` はプロジェクト内を参照します。

In [3]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    # データ・出力のパス（必要に応じて変更）
    data_dir = Path('/content/drive/MyDrive/fiber_analysis/data')
    output_dir = Path('/content/drive/MyDrive/fiber_analysis/output')
    config_dir = Path('/content/drive/MyDrive/fiber_analysis/params')
else:
    data_dir = PROJECT_ROOT / "data"
    output_dir = PROJECT_ROOT / "output"
    config_dir = PROJECT_ROOT / "params"
    print("ローカル実行のため Drive マウントはスキップしました。data / output はプロジェクト内を使用します。")

ローカル実行のため Drive マウントはスキップしました。data / output はプロジェクト内を使用します。


## 3. ケース実行（図化するケースの指定①・出力先: output/<ケース名>/）

**図化するケース**は、下のセルで **`CASE_NAME`** を指定します。

- **params/case_list.yml がある場合**: `CASE_NAME` に case_list の **id** を指定すると 1 件のみ、`None` または `""` で case_list の **全 id** を実行します。
- **case_list.yml が無い場合**: `CASE_NAME = "case_01"` のように **`params/cases/<CASE_NAME>.yml`** の stem で 1 件、`None` または `""` で **params/cases/*.yml をすべて**実行します。

結果は **`output/<ケース名>/`** に保存されます。ノートブックには 1 行サマリのみ表示し、詳細は output フォルダで確認してください。

In [4]:
from src.cli import run_single_case, run_all_cases

# 図化するケース: 1件なら CASE_NAME に id（case_list があればその id、なければ cases/<name>.yml の stem）。None または "" で全ケース
CASE_NAME = ""   # 例: "EF_2m_H" や "case_01" で 1 ケース / None で全ケース

try:
    if CASE_NAME and CASE_NAME.strip():
        results = run_single_case(
            CASE_NAME.strip(),
            output_dir=output_dir,
            config_dir=config_dir,
            data_dir=data_dir,
            verbose=False,
        )
        if len(results) > 0:
            r0 = results[0]
            print(f"完了: {output_dir / CASE_NAME.strip()} に保存 (kernel={r0.get('kernel','')} RMSE={r0['rmse']:.6f})")
    else:
        df = run_all_cases(output_dir, config_dir=config_dir, data_dir=data_dir, verbose=False)
        n_cases = df["case_id"].nunique() if not df.empty and "case_id" in df.columns else 0
        print(f"完了: {n_cases} ケースを {output_dir} に保存しました。")
except Exception as e:
    print(f"エラー: {e}")
    raise

ケース実行:  75%|███████▌  | 12/16 [00:01<00:00,  5.97it/s]窓幅 (4.0) が区間長 (2.0) を超えています (ratio=2.00)。 平滑化は区間外データに依存します。意図的か確認してください。
伸縮ひずみ |ε| > 1.0 のデータが 5 件あります。CSV全体 (EF-2m-H.csv)
ケース実行:  88%|████████▊ | 14/16 [00:02<00:00,  6.43it/s]窓幅 (4.0) が区間長 (2.0) を超えています (ratio=2.00)。 平滑化は区間外データに依存します。意図的か確認してください。
伸縮ひずみ |ε| > 1.0 のデータが 4 件あります。CSV全体 (EF-2m-V.csv)
ケース実行:  94%|█████████▍| 15/16 [00:02<00:00,  6.50it/s]伸縮ひずみ |ε| > 1.0 のデータが 8 件あります。CSV全体 (EF-4m-V.csv)
全欠損のひずみ列を除外します（線形補間不可）: Unnamed: 12
ケース実行: 100%|██████████| 16/16 [00:02<00:00,  6.39it/s]

完了: 16 ケースを /srv/shared_drive/DFOS-Element-Tests/output に保存しました。


## 4. 複数ケース比較（図化するケースの指定②・出力先: output/<output_subdir>/）

**図化するケース（比較に含めるケース）**は **`params/compare_groups.yml`** の **`groups`** で定義します（各グループは **`id`**・**`cases`**・**`output_subdir`** を持ちます）。

- **1 グループだけ**: 下のセルで **`GROUP_ID`** に実行したいグループの **`id`** を指定します（例: `"compare_2m_4m"`）。
- **全グループ**: **`GROUP_ID = None`** または **`GROUP_ID = ""`** にすると、yml に記載の **すべてのグループ**を順に実行します。

ログと結果表はノートブックには出さず、**`output/<output_subdir>/`** にのみ保存されます。確認は output フォルダで行ってください。

In [5]:
import yaml
from src.cli import run_compare

# 図化する比較グループ: 1件なら groups[].id を指定。None または "" で yml の全グループを実行
GROUP_ID = ""   # 例: "compare_2m_4m" で 1 グループ / None で全グループ

compare_yml = config_dir / "compare_groups.yml"
if not compare_yml.exists():
    print("compare_groups.yml がありません。")
else:
    if GROUP_ID and str(GROUP_ID).strip():
        _, out_dir = run_compare(
            compare_config=compare_yml,
            group_id=GROUP_ID.strip(),
            output_dir=output_dir,
            config_dir=config_dir,
            data_dir=data_dir,
            verbose=False,
        )
        print(f"比較結果は {out_dir} に保存しました。")
    else:
        with open(compare_yml, encoding="utf-8") as f:
            data = yaml.safe_load(f) or {}
        groups = data.get("groups") or []
        if not groups:
            print("compare_groups.yml に groups がありません。")
        else:
            for g in groups:
                gid = (g or {}).get("id")
                if gid:
                    run_compare(
                        compare_config=compare_yml,
                        group_id=gid,
                        output_dir=output_dir,
                        config_dir=config_dir,
                        data_dir=data_dir,
                        verbose=False,
                    )
            print(f"完了: {len(groups)} グループを {output_dir} に保存しました。")

比較準備（ケース実行・結果収集）: 100%|██████████| 2/2 [00:00<00:00, 20510.04it/s]


完了: 2 グループを /srv/shared_drive/DFOS-Element-Tests/output に保存しました。
